# 추천시스템 (Recommendation System)

- (초반) 고객이 별로 없을때 -> 콘텐츠 기반 필터링을 통해 추천시스템 (Content-Based Recommendation)
- 고객이 좀 쌓이면 user-based, item-based 추천시스템 -> 협업필터링 (Collaborative Filtering)

# 1. 콘텐츠 기반 필터링 (Content-Based-Recommendation)

- 값들을 모두 벡터로 변환해서 cos유사도를 기반으로 추천

In [1]:
# https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
# tmdb_5000_movies.csv

In [118]:
import pandas as pd

In [119]:
df = pd.read_csv("data/tmdb_5000_movies.csv")
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [120]:
movies_df = df[["id", "title", "genres", "vote_average", "vote_count", "popularity", "keywords", "overview"]]
pd.set_option("max_colwidth", 100)
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."


In [121]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [122]:
movies_df.isnull().sum()

id              0
title           0
genres          0
vote_average    0
vote_count      0
popularity      0
keywords        0
overview        3
dtype: int64

In [123]:
# 첫번째 행의 genres 열을 가져옴
movies_df.iloc[0]["genres"]
# 결과값 : 리스트 아닌 str 임

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [124]:
from ast import literal_eval

In [125]:
# str -> 리스트 형태로 변환
result = literal_eval(movies_df.iloc[0]["genres"])
result

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [126]:
# 전체 genres 열을 모두 str-> 리스트 형태로 변환
movies_df["genres"] = movies_df["genres"].apply(literal_eval)
movies_df["genres"]

C:\Users\user\AppData\Local\Temp\ipykernel_17068\2386163612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["genres"] = movies_df["genres"].apply(literal_eval)


0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
1            [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
2              [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...
                                                       ...                                                 
4798            [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]
4799                                       [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'...
4801                        

In [127]:
name_result = []
for genre_temp in movies_df["genres"] :
    name_list = []
    for temp in genre_temp :
        name = temp["name"]
        name_list.append(name)

    name_result.append(name_list)

movies_df["genres"] = name_result

C:\Users\user\AppData\Local\Temp\ipykernel_17068\3753552339.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["genres"] = name_result


In [128]:
movies_df["genres"]

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [129]:
# 장르를 ['Action', 'Adventure', 'Fantasy', 'Science Fiction']
# -> 'Action Adventure Fantasy Science Fiction' 이렇게 변경하고 싶음
data_result = []
for temp in movies_df["genres"] :
    data = (' ').join(temp)
    data_result.append(data)

movies_df["genres_literal"] = data_result
movies_df["genres_literal"]


C:\Users\user\AppData\Local\Temp\ipykernel_17068\1281784647.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["genres_literal"] = data_result


0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

## CountVectorizer

In [131]:
from sklearn.feature_extraction.text import CountVectorizer

# ngram_range
# Action : 1, Comedy : 2, TV : 3, Movie : 4, Science Fiction 은
# Science Fiction 은 한 장르임 -> 장르가 안나올 수 있음
# ngram_range=(1,2) : 단어 1개짜리 다 뽑고, 2개 연결해서도 뽑기 (Action도 뽑고, Action Comedy도 뽑고)
# min_df = 1 : 이 단어가 최소 1 번이라도 나온 경우를 쓰겠다 -> Action Comedy 는 안나옴!
count_vect = CountVectorizer(min_df=1, ngram_range=(1,2))
genre_vect = count_vect.fit_transform(movies_df["genres_literal"])
count_vect.vocabulary_

{'action': 0,
 'adventure': 16,
 'fantasy': 124,
 'science': 232,
 'fiction': 138,
 'action adventure': 1,
 'adventure fantasy': 24,
 'fantasy science': 135,
 'science fiction': 233,
 'fantasy action': 125,
 'crime': 64,
 'adventure crime': 20,
 'drama': 90,
 'thriller': 234,
 'action crime': 4,
 'crime drama': 68,
 'drama thriller': 106,
 'adventure science': 29,
 'animation': 33,
 'family': 109,
 'animation family': 38,
 'fantasy family': 130,
 'action science': 12,
 'adventure action': 17,
 'action thriller': 13,
 'thriller crime': 238,
 'western': 265,
 'adventure western': 32,
 'adventure family': 23,
 'family fantasy': 115,
 'fiction action': 139,
 'action fantasy': 7,
 'comedy': 44,
 'action comedy': 3,
 'comedy science': 59,
 'adventure drama': 22,
 'drama action': 91,
 'romance': 214,
 'drama romance': 104,
 'romance thriller': 228,
 'thriller action': 235,
 'fiction thriller': 150,
 'adventure thriller': 30,
 'fantasy adventure': 126,
 'family adventure': 111,
 'adventure com

In [132]:
genre_vect.shape

(4803, 276)

In [133]:
genre_vect_array = genre_vect.toarray()
genre_vect_array

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 276))

In [135]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_vect_array, genre_vect_array)
genre_sim

# 뭐가 좋을까?
#       A   B   C   D
# A     1  0.3 0.8 0.7  -> '본인 제외 가장 높은 유사도를 갖고있는 걸 추천해주면 됨'
# B
# C
# D

# -> C, D, B 순


array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]], shape=(4803, 4803))

In [136]:
genre_sim[0].argsort()[::-1]
# -> 0번 영화와 가장 유사한 영화는 46번 영화/ 그다음은 3494 영화

array([   0,   46, 3494, ..., 3600, 3569, 4799], shape=(4803,))

In [137]:
print(movies_df["title"].iloc[0])
print(movies_df["title"].iloc[46])

Avatar
X-Men: Days of Future Past


In [138]:
title_movie = df[df["title"] == "X-Men: Days of Future Past"]
title_movie.index.values

array([46])

In [139]:
def find_sim_movie(df, sorted_index, title_name, top_n=10) :
    title_movie = df[df["title"] == title_name]
    title_index = title_movie.index.values

    similar_index = sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)

    return df.iloc[similar_index]

In [140]:
genre_sim.argsort()[:, ::-1]

array([[   0,   46, 3494, ..., 3600, 3569, 4799],
       [   1,  129,   12, ..., 4754, 4753, 3829],
       [1740, 1542,    2, ..., 4757, 4758, 4795],
       ...,
       [4800, 3809, 3186, ..., 4771, 3640,   41],
       [4802, 4801, 4800, ...,    2,    1,    0],
       [4698, 4524, 3891, ...,    2,    1, 4800]], shape=(4803, 4803))

In [145]:
genre_sim_sorted = genre_sim.argsort()[:, ::-1]

movie_result = find_sim_movie(df, genre_sim_sorted, "Titanic", 10)

In [146]:
movie_result["title"]

25                   Titanic
2927        Cruel Intentions
1154    Fifty Shades of Grey
2667        Fatal Attraction
2952                       O
1311              Angel Eyes
4397          Alone With Her
3700                   Bound
2818                 Witness
2128         Man of the Year
Name: title, dtype: object

------------------------

- keywords

In [147]:
# keywords도 확인
movies_df.iloc[0]["keywords"]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [148]:
movies_df["keywords"] = movies_df["keywords"].apply(literal_eval)
movies_df["keywords"]

C:\Users\user\AppData\Local\Temp\ipykernel_17068\3795952246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["keywords"] = movies_df["keywords"].apply(literal_eval)


0       [{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...
1       [{'id': 270, 'name': 'ocean'}, {'id': 726, 'name': 'drug abuse'}, {'id': 911, 'name': 'exotic is...
2       [{'id': 470, 'name': 'spy'}, {'id': 818, 'name': 'based on novel'}, {'id': 4289, 'name': 'secret...
3       [{'id': 849, 'name': 'dc comics'}, {'id': 853, 'name': 'crime fighter'}, {'id': 949, 'name': 'te...
4       [{'id': 818, 'name': 'based on novel'}, {'id': 839, 'name': 'mars'}, {'id': 1456, 'name': 'medal...
                                                       ...                                                 
4798    [{'id': 5616, 'name': 'united states–mexico barrier'}, {'id': 33649, 'name': 'legs'}, {'id': 162...
4799                                                                                                     []
4800    [{'id': 248, 'name': 'date'}, {'id': 699, 'name': 'love at first sight'}, {'id': 2398, 'name': '...
4801                        

In [149]:
keywords_result = []
for keyword_temp in movies_df["keywords"] :
    keyword_list = []
    for temp in keyword_temp :
        keyword = temp["name"]
        keyword_list.append(keyword)
    
    keywords_result.append(keyword_list)

movies_df["keywords"] = keywords_result

C:\Users\user\AppData\Local\Temp\ipykernel_17068\2549471746.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["keywords"] = keywords_result


In [150]:
keyword_data_result = []
for temp in movies_df["keywords"] :
    data = (' ').join(temp)
    keyword_data_result.append(data)

movies_df["keywords_literal"] = keyword_data_result
movies_df["keywords_literal"]

C:\Users\user\AppData\Local\Temp\ipykernel_17068\2304701140.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["keywords_literal"] = keyword_data_result


0       culture clash future space war space colony society space travel futuristic romance space alien ...
1       ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck s...
2                          spy based on novel secret agent sequel mi6 british secret service united kingdom
3       dc comics crime fighter terrorist secret identity burglar hostage drama time bomb gotham city vi...
4       based on novel mars medallion space travel princess alien steampunk martian escape edgar rice bu...
                                                       ...                                                 
4798                                         united states–mexico barrier legs arms paper knife guitar case
4799                                                                                                       
4800                                    date love at first sight narration investigation team postal worker
4801                        

In [151]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=1, ngram_range=(1,2))
keywords_vect = count_vect.fit_transform(movies_df["keywords_literal"])
count_vect.vocabulary_

{'culture': 9037,
 'clash': 7010,
 'future': 15032,
 'space': 34485,
 'war': 40021,
 'colony': 7433,
 'society': 34214,
 'travel': 38236,
 'futuristic': 15087,
 'romance': 31412,
 'alien': 1077,
 'tribe': 38408,
 'planet': 27732,
 'cgi': 6161,
 'marine': 22347,
 'soldier': 34275,
 'battle': 3232,
 'love': 21496,
 'affair': 548,
 'anti': 1760,
 'power': 28328,
 'relations': 30072,
 'mind': 23178,
 'and': 1426,
 'soul': 34431,
 '3d': 94,
 'culture clash': 9039,
 'clash future': 7012,
 'future space': 15075,
 'space war': 34525,
 'war space': 40145,
 'space colony': 34494,
 'colony society': 7442,
 'society space': 34240,
 'space travel': 34523,
 'travel futuristic': 38253,
 'futuristic romance': 15094,
 'romance space': 31433,
 'space alien': 34487,
 'alien tribe': 1148,
 'tribe alien': 38409,
 'alien planet': 1122,
 'planet cgi': 27737,
 'cgi marine': 6165,
 'marine soldier': 22373,
 'soldier battle': 34281,
 'battle love': 3254,
 'love affair': 21498,
 'affair anti': 550,
 'anti war': 

In [152]:
keywords_vect.shape

(4803, 41554)

In [153]:
keywords_vect_array = keywords_vect.toarray()
keywords_vect_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 41554))

In [154]:
from sklearn.metrics.pairwise import cosine_similarity

keyword_sim = cosine_similarity(keywords_vect_array, keywords_vect_array)
keyword_sim

array([[1.        , 0.01628008, 0.        , ..., 0.02614435, 0.        ,
        0.        ],
       [0.01628008, 1.        , 0.        , ..., 0.03277368, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02614435, 0.03277368, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]], shape=(4803, 4803))

In [165]:
def find_sim_keywords(df, sorted_index, title_name, top_n=10) :
    title_movie = df[df["title"] == title_name]
    title_index = title_movie.index.values

    similar_index = sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)

    return df.iloc[similar_index]

In [166]:
keyword_sim_sorted = keyword_sim.argsort()[:, ::-1]
movie_result = find_sim_keywords(df, keyword_sim_sorted, "Superman", 10)

In [167]:
keyword_sim_sorted

array([[   0, 2403,  278, ..., 4789, 4790, 4791],
       [   1,   12, 2875, ...,   57,   58, 4801],
       [   2,   11,  147, ...,   41,   42, 4775],
       ...,
       [4800, 2674, 2108, ..., 4780,   45, 4788],
       [4802, 4801, 4800, ...,    2,    1,    0],
       [4802, 2000, 3968, ...,    2,    1, 4800]], shape=(4803, 4803))

In [168]:
movie_result['title']

813                               Superman
10                        Superman Returns
1296                          Superman III
14                            Man of Steel
870                            Superman II
65                         The Dark Knight
9       Batman v Superman: Dawn of Justice
2433      Superman IV: The Quest for Peace
203                                     X2
511                                  X-Men
Name: title, dtype: object